In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import requests
import json

In [2]:
file_path = "/Users/thesavage/Global_Key/Data/final_tickers.xlsx"

In [3]:
companies_df = pd.read_excel(file_path)

In [4]:
companies_df = companies_df.astype(str)
companies_df["SEC ID"] = companies_df["SEC ID"].apply(lambda x: x.zfill(10))

In [5]:
companies_df

,Ticker,Name,SEC ID,CountryCode
0,nan,nan,0001448983,CN
1,000029.SZ,ShenZen Special Economic Zone Real Estate,0980000109,CN
2,0123.HK,Yuexiu Property,0980000104,HK
3,0RDM.IL,ABN AMRO GROUP NV ABN AMRO GROU,0001699968,NL
4,5021.T,"Cosmo Energy Holdings Co., Ltd.",0001668393,JP
...,...,...,...,...
7412,ZURVY,ZURICH INSURANCE GROUP,0001277799,CH
7413,ZVO,Zovio Inc,0001305323,US
7414,ZY,"Zymergen, Inc.",0001645842,US
7415,ZYME,Zymeworks,0001403752,CA


In [6]:
us_companies = companies_df.loc[companies_df["CountryCode"] == "US"]
us_companies.tail(50)

,Ticker,Name,SEC ID,CountryCode
7337,XPL,Solitario Exploration & Royalty Corp,0000917225,US
7338,XPO,"Express-1 Expedited Solutions, Inc.",0001166003,US
7339,XRAY,DENTSPLY INTERNATIONAL INC,0000818479,US
7341,XRX,Xerox Corp,0000108772,US
7342,XSPA,"XpresSpa Group, Inc.",0001410428,US
7344,XTNT,"Xtant Medical Holdings, Inc.",0001453593,US
7347,XYL,Xylem Inc.,0001524472,US
7348,Y,Alleghany Corp,0000775368,US
7353,YELP,Yelp Incorporated,0001345016,US
7354,YETI,YETI Holdings,0001670592,US


In [7]:
header = {"User-Agent": "cflan1278@gmail.com"} # replace with your email

In [8]:
def get_cik(ticker, df): 
    """
    get cik from df given ticker
    
    """
    cik = df[df["Ticker"] == ticker]["SEC ID"].values[0]
    return cik

In [9]:
def get_endpoint(ticker, df, endpoint = "submissions"):
    '''
    Creates an endpoint given a ticker to request SEC API.
    Default is submissions look at sec api docs for other endpoint
    types
    '''
    
    cik = get_cik(ticker, df) # get cik from ticker
    endpoint_url = f"https://data.sec.gov/{endpoint}/CIK{cik}.json" # create a URL to request in future 
    return cik, endpoint_url

In [34]:
# example of get_endpoint()
cik, endpoint = get_endpoint("YETI", us_companies)
print(cik)
endpoint

0001670592


'https://data.sec.gov/submissions/CIK0001670592.json'

In [35]:
def filings_df(endpoint):
    '''
    This takes in an endpoint url (use get_endpoint to make url)
    requests the endpoint to get company filings.
    Creates a dataframe of the company filings
    '''
    
    edgar_json = requests.get(endpoint, headers = header).json() # get request 
                                                                 #(returns json (basically a dictionary))
    
    company_filings = edgar_json["filings"]["recent"] # access the recent files that the API has on deck
    filings_df = pd.DataFrame(company_filings)
    return filings_df

In [36]:
# example of filings_df()
yeti_files = filings_df(endpoint)
yeti_files

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001104659-24-026134,2024-02-21,2024-02-16,2024-02-21T17:42:18.000Z,,4,,,,10752,0,0,xslF345X05/tm246135-8_4seq1.xml,OWNERSHIP DOCUMENT
1,0001104659-24-026133,2024-02-21,2024-02-16,2024-02-21T17:40:56.000Z,,4,,,,11835,0,0,xslF345X05/tm246135-7_4seq1.xml,OWNERSHIP DOCUMENT
2,0001104659-24-026132,2024-02-21,2024-02-16,2024-02-21T17:39:39.000Z,,4,,,,10742,0,0,xslF345X05/tm246135-6_4seq1.xml,OWNERSHIP DOCUMENT
3,0001104659-24-026131,2024-02-21,2024-02-16,2024-02-21T17:39:30.000Z,,4,,,,5252,0,0,xslF345X05/tm246135-5_4seq1.xml,OWNERSHIP DOCUMENT
4,0001104659-24-025530,2024-02-20,2024-02-16,2024-02-20T16:17:04.000Z,34,8-K,001-38713,24653541,5.02,191783,1,1,tm246714d1_8k.htm,FORM 8-K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0001144204-18-016750,2018-03-26,,2018-03-23T17:39:01.000Z,NE,RW,333-212379,18711196,,28470,0,0,tv489423_rw.htm,FORM RW
432,0001047469-16-014127,2016-07-01,,2016-07-01T15:50:37.000Z,33,S-1,333-212379,161746832,,20212013,0,0,a2229052zs-1.htm,S-1
433,0000912057-16-000619,2016-06-24,,2016-06-24T17:03:49.000Z,33,DRS/A,377-01300,161731674,,14346828,0,0,filename1.htm,
434,0000912057-16-000603,2016-06-07,,2016-06-06T21:14:09.000Z,33,DRS/A,377-01300,161699847,,4237826,0,0,filename1.htm,


In [37]:
def certain_file(file_type, df):
    """
    Filter down the filing dataframe to just the desired files
    """
    
    return df[df["form"] == file_type] 

In [38]:
def create_request_link(cik, access_num, file_name):
    """
    Create a URL we can use to request access to one of the 
    comapnies filings.
    """
    
    
    return f"https://www.sec.gov/Archives/edgar/data/{cik}/{access_num}/{file_name}" # formatted-string


In [39]:
def access_s_1(ticker):
    """
    Filter down a companies filings dataframe to only
    include s-1 documents. Then adds a new column to the 
    dataframe of each of the files request url
    
    
    """
    
    
    
    cik, endpoint = get_endpoint(ticker, us_companies)
    filings = filings_df(endpoint)
    s_1 = filings[filings["form"] == "S-1"].copy().sort_values(by = "filingDate", ascending = False)
    
    if s_1.shape[0] == 0: # If the company doesnt have an s-1 doc return nothing
         return 
        
    s_1["accessionNumber"] = s_1.apply(lambda row: row["accessionNumber"].replace("-", ""), axis = 1) # cleans the accession numbers
    
    # adds a column of endpoints/file URLs to request 
    s_1["file_endpoint"] = s_1.apply(lambda row: create_request_link(cik, row['accessionNumber'], row['primaryDocument']), axis=1)
    request_url = s_1["file_endpoint"].values[0]
    print(request_url)
    return requests.get(request_url, headers = header).content




In [40]:
# example access_s_1()
yeti_s_1 = access_s_1("YETI")
yeti_s_1

https://www.sec.gov/Archives/edgar/data/0001670592/000104746919006054/a2239334zs-1.htm


b'<DOCUMENT>\n<TYPE>S-1\n<SEQUENCE>1\n<FILENAME>a2239334zs-1.htm\n<DESCRIPTION>S-1\n<TEXT>\n<HTML>\n<HEAD>\n</HEAD>\n<BODY BGCOLOR="#FFFFFF" LINK=BLUE  VLINK=PURPLE>\n<BR>\n<P><FONT SIZE=3 >\nUse these links to rapidly review the document<BR>\n<A HREF="#bg79201_table_of_contents">  TABLE OF CONTENTS</A> <BR>\n<A HREF="#fa79201_index_#160;to_#160;financials">  INDEX&#160;TO&#160;FINANCIALS</A><BR></font>\n</P>\n\n<P style="font-family:times;text-align:justify"><FONT SIZE=2>\n\n<!-- COMMAND=ADD_BASECOLOR,"#000000" -->\n\n\n\n\n<!-- COMMAND=ADD_DEFAULTFONT,"font-family:times;" -->\n\n\n\n\n<!-- COMMAND=ADD_TABLESHADECOLOR,"#CCEEFF" -->\n\n\n\n\n<!-- COMMAND=ADD_STABLERULES,"border-bottom:solid #000000 1.0pt;" -->\n\n\n\n\n\n<!-- COMMAND=ADD_DTABLERULES,"border-bottom:double #000000 2.25pt;" -->\n\n\n\n\n<!-- COMMAND=ADD_SCRTABLERULES,"border-bottom:solid #000000 1.0pt;margin-bottom:0pt;" -->\n\n\n\n\n<!-- COMMAND=ADD_DCRTABLERULES,"border-bottom:double #000000 2.25pt;margin-bottom:0pt;" -

In [41]:
# found this on stack overflow just cleans the text more.
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

In [42]:
def split_pages(sec_doc): 
    """
    Takes in a single document/filings and splits it into separate
    pages depedning on a certain page delimeter
    
    Reuturns a list of pages each of which is in HTML form
    """
    
    
    soup = BeautifulSoup(sec_doc, "html.parser") # creates a parsing object (of page passed in)
    text = soup.find("text")
    
    if text is None:
        text = soup
    
    # find page breaks
    page_breaks = text.find_all("hr", {"noshade": True}) + text.find_all("hr", {"width": "100%"}) + text.find_all("hr", {"align": "left"}) + text.find_all("div", {"id" : "pgbrk"}) + text.find_all("hr") 


    # make a list of page breaks
    page_breaks = [str(page_break) for page_break in page_breaks]
    
    if len(page_breaks) > 0:
        #create pattern
        regex_pattern = "|".join(map(re.escape, page_breaks))

        #split document along breaks (this stores pages in a list)
        split_filing = re.split(regex_pattern, str(text))
    
    return split_filing




In [43]:
# example of split_pages
split_filing = split_pages(yeti_s_1) # must use access_s_1 to access file before using this function
len(split_filing)

247

In [44]:
def clean_filing(split_filing):
    '''
    Takes in a filing that is split (use split_filing function to do so)
    
    create pages for the documents (keys of dictionary are the "page number")
    saves the table and normalized text of each corresponding page
    
    '''
    # this is our filing when populated
    # keys = page #, values = cleaned page content
    cleaned_filing = {}
    
    for index, page in enumerate(split_filing):
        page_num = index # create page number 
        
        # clean and normalize pages of filing
        # then store in above dictionary
        
        soup = BeautifulSoup(page, "html5") # adds tags and restructure html
        page_text = soup.html.body.get_text(" ", strip = True)
        
        normalized_text = restore_windows_1252_characters(unicodedata.normalize('NFKD', page_text)).replace("  ", " ")
        cleaned_filing[page_num] = normalized_text
        
    return cleaned_filing
    
            

In [45]:
# example clean_filing()
cleaned_file = clean_filing(split_filing)
cleaned_file

{0: 'Use these links to rapidly review the document TABLE OF CONTENTS INDEX TO FINANCIALS Table of Contents As filed with the Securities and Exchange Commission on November 4, 2019 Registration No. 333- UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549',
 1: 'FORM S-1 REGISTRATION STATEMENT UNDER THE SECURITIES ACT OF 1933 YETI Holdings, Inc. (Exact name of registrant as specified in its charter) Delaware (State or other jurisdiction of incorporation or organization) 3949 (Primary Standard Industrial Classification Code Number) 45-5297111 (I.R.S. Employer Identification Number)',
 2: "7601 Southwest Parkway Austin, Texas 78735 (512) 394-9384 (Address, including zip code, and telephone number, including area code, of registrant's principal executive offices)",
 3: 'Matthew J. Reintjes President and Chief Executive Officer, Director 7601 Southwest Parkway Austin, Texas 78735 (512) 394-9384 (Name, address, including zip code, and telephone number, including area code

In [46]:
# search for certain words and use regex expressions to find the managment page
search_words = {
    "key_words" : ["MANAGEMENT", "Chief Executive Officer", "Executive Officers", "Position", "Chief", "Name Age", "Director"],
}


In [47]:
def find_page(filing_dict, key_dict = 
              {
    "key_words" : ["MANAGEMENT", "Chief Executive Officer", "Executive Officers", "Position", "Chief", "Name Age", "Director"],
    "regex" : []
                  
}
             ):
    
    '''
    takes in a filing dictionary (use clean_filing() to make this)
    
    searches through the document for the keywords and returns the 
    page number of the page with the most key word appearances
    
    '''
    
    # dictionary whose key is page number
    # value is how may desired matches were on page
    num_matches = {}
    
    for page_num, content in filing_dict.items():
        num_matches[page_num] = 0
        key_words = key_dict["key_words"]
        # for each key word in the key word list
        # this list has the count for each word on each list
        counter = [content.count(word) for word in key_words]
        num_matches[page_num] += sum(counter) # assign the page number (key)
                                              # the num of word matches on that page
            
        # assuming the page with the most matches is the page we want to scrape
        
    return max(num_matches, key = num_matches.get)
        
    

In [48]:
# example
page = find_page(cleaned_file)
page

117

In [49]:
def find_table(page_num, doc_split):
    """
    takes in a page number and a split up filing (list) use
    split_filing() to create
    
    Given the matched page we will search this page for
    the executive managment table. We will clean and parse the
    table and return a df of that table.
    """
    
    soup = BeautifulSoup(doc_split[page_num], "html.parser") # parser to parese html of page
    
    # get rows for the table
    tables = soup.find_all("table") # find all the tables on the page 
    
    first_table = tables[0]
    table_rows = first_table.find_all("tr") # find all the rows in the table
    table_header = first_table.find("th")
    
    # parse/clean table 
    
    # gets the data stored in each of the tr tags
    parsed_table = [
    [data.get_text(strip = True) for data in row.find_all("td")]
    for row in table_rows
    ]
    
    print(parsed_table)
    print("\n")

    
    # gets rid of all elements that are empty
    cleaner_table = [
        [item for item in array if item not in ["", '\u200b']]
        for array in parsed_table
    ]
    
    print(cleaner_table)
    print("\n")

    # gets rid of all the lists that contain nothing
    cleaned_table = [array for array in cleaner_table if len(array) == 3]

    print(cleaned_table)
    print("\n")
    
    # create dictionary that we can turn into a dataframe
    to_df_dict = {key : [] for key in cleaned_table[0]} # keys are the first item (array) in "cleaned table"
    
    # populate the dictionary
    keys = ["Name ", "Age", "Position"]

    to_df = {key : [] for key in keys}

    contents = cleaned_table[1:] if cleaned_table[0][1] == "Age" else cleaned_table

    
    # populates the dictionary that we will then turn into a dataframe
    # each array in contents is like a table row
    for array in contents:
        for index, item in enumerate(array):
            to_df[keys[index]].append(item)

    
    return pd.DataFrame(to_df)


In [50]:
find_table(page, split_filing)

[['', '', '', '', '', ''], [], ['Matthew J. Reintjes', '', '', '44', '', 'President and CEO, Director'], ['Paul C. Carbone', '', '', '53', '', 'Senior Vice President and Chief Financial Officer'], ['Bryan C. Barksdale', '', '', '48', '', 'Senior Vice President, General Counsel and Secretary'], ['Hollie S. Castro', '', '', '50', '', 'Senior Vice President of Talent'], ['Robert O. Murdock', '', '', '48', '', 'Senior Vice President, Direct-to-Consumer and Managing Director, International'], ['Kirk A. Zambetti', '', '', '51', '', 'Senior Vice President of Sales'], ['David L. Schnadig', '', '', '55', '', 'Chair of the Board and Director'], ['Mary Lou Kelley', '', '', '59', '', 'Director'], ['Jeffrey A. Lipsitz', '', '', '54', '', 'Director'], ['Dustan E. McCoy', '', '', '70', '', 'Director'], ['Michael E. Najjar', '', '', '52', '', 'Director'], ['Roy J. Seiders', '', '', '42', '', 'Director'], ['Robert K. Shearer', '', '', '67', '', 'Director']]


[[], [], ['Matthew J. Reintjes', '44', 'Pre

,Name,Age,Position
0,Matthew J. Reintjes,44,"President and CEO, Director"
1,Paul C. Carbone,53,Senior Vice President and Chief Financial Officer
2,Bryan C. Barksdale,48,"Senior Vice President, General Counsel and Sec..."
3,Hollie S. Castro,50,Senior Vice President of Talent
4,Robert O. Murdock,48,"Senior Vice President, Direct-to-Consumer and ..."
5,Kirk A. Zambetti,51,Senior Vice President of Sales
6,David L. Schnadig,55,Chair of the Board and Director
7,Mary Lou Kelley,59,Director
8,Jeffrey A. Lipsitz,54,Director
9,Dustan E. McCoy,70,Director


In [51]:
def exec_df(ticks):
    '''
    Loop through all the compaines create a massive df of Name age and. position
    '''
    concat_list = []
    counter = 0
    for tick in ticks:
        try:
            s1_file = access_s_1(tick)
            print(tick)
            s1_split = split_pages(s1_file)
            s1_cleaned = clean_filing(s1_split)

            page = find_page(s1_cleaned)
            table = find_table(page, s1_split)
            table["company"] = tick

            concat_list.append(table)
            counter += 1
        except Exception as e:
            print("No File for: " + tick) 
            continue

    big_df = pd.concat(concat_list)

    print(counter)
        
    return big_df


In [52]:
exec_df = exec_df(us_companies["Ticker"].values[:100])

A
No File for: A
https://www.sec.gov/Archives/edgar/data/0001675149/000119312517012427/d309876ds1.htm
AA


KeyboardInterrupt: 

In [33]:
exec_df

,Name,Age,Position,company
10,Reed B. Rayman,31,Director,ADT
11,Eric L. Press,51,Director,ADT
12,Lee J. Solomon,45,Director,ADT
13,Stephanie Drescher,44,Director,ADT
14,Brett Watson,37,Director,ADT
15,David Ryan,47,Director,ADT
0,Mark S. Heaney,53,"Chairman of the Board, President and Chief Exe...",ADUS
1,Francis J. Leonard,53,"Chief Financial Officer, Vice President and Se...",ADUS
2,Darby Anderson,43,Vice President of Home & Community Services of...,ADUS
3,Sharon Rudden,49,Vice President of Home Health Services of Addu...,ADUS
